# Data Processing

In [2]:
import cv2,os

# Chargement des données dans categories
data_path=r'dataset'
categories=os.listdir(data_path)
# Liste des labels associés aux dossiers "with mask" and "without mask"
labels=[i for i in range(len(categories))]

# On crée un dictionnaire avec categories et labels
label_dict=dict(zip(categories,labels)) 

print(label_dict)
print(categories)
print(labels)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [3]:
img_size=100
# Création de la cible et du jeu de données
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
     
     # Chargement image par image     
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            # Transformation de l'image en gris
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            # redimensionnement de l'image 100x100
            resized=cv2.resize(gray,(img_size,img_size))
            
            # Mise dans deux listes différentes de la cible et du jeu de donneées
            data.append(resized)
            target.append(label_dict[category])

        # Exception pour des images altérées
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [4]:
import numpy as np

data=np.array(data)/255.0
# Redimensionnement de l'image de 2D en 4D
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

# On classe l'image soit en "whithout mask" ou "with mask"
new_target=np_utils.to_categorical(target)

In [5]:
# Enregistrement des images
np.save('data',data)

# Enregistrement des labels "whithout mask" et "with mask" correspondant aux images
np.save('target',new_target)

# CNN

In [7]:
import numpy as np
# Chargement des images et des labels  "whithout mask" et "with mask"
data=np.load(r'data.npy')
target=np.load(r'target.npy')

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

# Instance ajoutant les couches CNN en séquence
model=Sequential()

# Première couche avec 200 kernels suivi par Relu et les couches MaxPooling 
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Deuxième couche avec 100 kernels suivi par Relu et les couches MaxPooling 
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Une couche Flatten pour empiler les couches de convolution  
model.add(Flatten())
model.add(Dropout(0.5))

# Activation de la couche relu
model.add(Dense(50,activation='relu'))
#Couche dense de 64 neurones
model.add(Dense(2,activation='softmax'))
# Cette dernière couche sépare en deux catégories les données 

# Affichage de l'accuracy durant l'apprentissage
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
from sklearn.model_selection import train_test_split

# Fractionnement du jeu de données (90% de jeu d'entraînement, 10% jeu de test)

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [10]:
# Enregistrement des meilleurs modèles
# Vérification des répertoires des Epoch
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
27/27 [==============================] - 141s 5s/step - loss: 0.7634 - accuracy: 0.5242 - val_loss: 0.6340 - val_accuracy: 0.5787
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
27/27 [==============================] - 134s 5s/step - loss: 0.5405 - accuracy: 0.7284 - val_loss: 0.5409 - val_accuracy: 0.7546
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/20
27/27 [==============================] - 142s 5s/step - loss: 0.3522 - accuracy: 0.8504 - val_loss: 0.3134 - val_accuracy: 0.8611
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/20
27/27 [==============================] - 130s 5s/step - loss: 0.2093 - accuracy: 0.9228 - val_loss: 0.2559 - val_accuracy: 0.9028
INFO:tensorflow:Assets written to: model-004.model\assets
Epoch 5/20
27/27 [==============================] - 136s 5s/step - loss: 0.1237 - accuracy: 0.9668 - val_loss: 0.2116 - val_accuracy: 0.9213
INFO:tensorflow:Assets written to: model-005.model\assets
Epoch

# Mask Detection

In [3]:
from keras.models import load_model
import cv2
import numpy as np

In [4]:
from pygame import mixer 
import time

happy = cv2.imread('./care-emoji-open-eye-fb-2.png')

def son():
    # Création de l'instance
    mixer.init() 
    # Chargement de la musique
    mixer.music.load("radar1.wav") 
    # Réglage du volume
    mixer.music.set_volume(0.7) 
    # Lancer la musique
    mixer.music.play() 
    time.sleep(10)
    mixer.music.stop()
    
# Afficher l'emoji
def afficher_emo():
    img = cv2.imread('thumbs-up-4007573_640.png',0)
    cv2.imshow('image',img)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()   

In [12]:
# Chargement du meilleur modèle
model = load_model(r'model-005.model')
source=cv2.VideoCapture(2)

# Dictionnaire des labels
labels_dict={0:'MASK',1:'NO MASK'}
# Dictionnaire des RBG 
color_dict={0:(0,255,0),1:(0,0,255)}

# Cascade pour la détection d'image
face_clsfr=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
video_capture = cv2.VideoCapture(0)

while(True):
    # Capture de la vidéo en image ou frame
    ret, img = video_capture.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

     # Lancement du modèle pour chaque face détectée
    for (x,y,w,h) in faces:
         # Stockage de l'image ou face
        face_img=gray[y:y+w,x:x+w]
        # Redimensionnement de l'image en 100x100
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        # Convertion de l'image 4D dans un tableau(array)
        reshaped=np.reshape(normalized,(1,100,100,1))
         # Chargement du modèle pour une prédiction
        result=model.predict(reshaped)
        # Résultat de la prédiction (soit label "whithout mask" soit label "with mask")
        label=np.argmax(result,axis=1)[0]
        
        # Condition avec masque et sans masque   
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        # Son ou emoji selon le label
        if labels_dict[label] == 'NO MASK':
            son()
        else:
            afficher_emo()
            
    # Visionnage de la caméra en direct         
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
# Destruction de la fenêtre et fermeture de la caméra         
cv2.destroyAllWindows()
video_capture.release()
source.release()